In [84]:
import os
import struct
import numpy as np

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

In [56]:

def read(dataset = "training", path = "."):
   
    if dataset is "training":
        fname_img = os.path.join(path, 'train-images.idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels.idx1-ubyte')
    elif dataset is "testing":
        fname_img = os.path.join(path, 't10k-images.idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels.idx1-ubyte')
    

    # Load everything in some numpy arrays
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)

    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols)

    get_img = lambda idx: (lbl[idx], img[idx])

    # Create an iterator which returns each image in turn
    for i in range(len(lbl)):
        yield get_img(i)



In [62]:
train_data= read(dataset = "training", path = "/home/farzad/Desktop/CUrrent WEEK/CI project/MNIST/")
test_data = read(dataset = "training", path = "/home/farzad/Desktop/CUrrent WEEK/CI project/MNIST/")
train_data=list(train_data)
test_data=list(test_data)

In [68]:
images_train = [train_data[i][1].reshape(28*28) for i in range(5000) ]
labels_train = [train_data[i][0] for i in range(5000) ]
images_test = [test_data[i][1].reshape(28*28) for i in range(5000) ]
labels_test = [train_data[i][0] for i in range(5000) ]


In [109]:
X_train = np.array(images_train, dtype=np.float32)
X_test = np.array(images_test, dtype=np.float32)

y_train = np.array(labels_train)
y_test = np.array(labels_test)

X_train/=255
X_test/=255

In [107]:
# Y_train.shape

(5000,)

In [111]:
number_of_classes = 10

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)


In [112]:
model = Sequential()

model.add(Dense(512, input_dim=784))
# An "activation" is just a non-linear function applied to the output
# of the layer above. Here, with a "rectified linear unit",
# we clamp all values below 0 to 0.
model.add(Activation('relu'))
# Dropout helps protect the model from memorizing or "overfitting" the training data
model.add(Dropout(0.2))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(10))
# This special "softmax" activation among other things,
# ensures the output is a valid probaility distribution, that is
# that its values are all non-negative and sum to 1.
model.add(Activation('softmax'))

In [113]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [114]:
size = int(len(X_train) * 0.8)

train_x, val_x = X_train[:size], X_train[size:]
train_y, val_y = Y_train[:size], Y_train[size:]

In [115]:
Y_train.shape

(5000, 10)

In [117]:
model.fit(train_x, train_y, batch_size=128, epochs=5, validation_data=(val_x, val_y))

Train on 4000 samples, validate on 1000 samples
Epoch 1/5
4000/4000 [==============================] - 2s 503us/step - loss: 0.0744 - acc: 0.9795 - val_loss: 0.2603 - val_acc: 0.9340
Epoch 2/5
4000/4000 [==============================] - 2s 514us/step - loss: 0.0510 - acc: 0.9852 - val_loss: 0.2122 - val_acc: 0.9400
Epoch 3/5
4000/4000 [==============================] - 2s 555us/step - loss: 0.0358 - acc: 0.9895 - val_loss: 0.2357 - val_acc: 0.9440
Epoch 4/5
4000/4000 [==============================] - 2s 500us/step - loss: 0.0283 - acc: 0.9908 - val_loss: 0.2443 - val_acc: 0.9450
Epoch 5/5
4000/4000 [==============================] - 2s 432us/step - loss: 0.0218 - acc: 0.9955 - val_loss: 0.2553 - val_acc: 0.9510


In [118]:
score = model.evaluate(X_test, Y_test)
print()
print('Test accuracy: ', score[1])

5000/5000 [==============================] - 1s 185us/step

Test accuracy:  0.9896


In [120]:
predictions = model.predict_classes(X_test)

predictions = list(predictions)

In [ ]:
# from matplotlib import pyplot
# import matplotlib as mpl
# def show(image):
#     fig = pyplot.figure()
#     ax = fig.add_subplot(1,1,1)
#     imgplot = ax.imshow(image, cmap=mpl.cm.Greys)
#     imgplot.set_interpolation('nearest')
#     ax.xaxis.set_ticks_position('top')
#     ax.yaxis.set_ticks_position('left')
#     pyplot.show()